In [70]:
import xgboost as xgb
import alpaca_trade_api as tradeapi
from finta import TA
import pickle

In [72]:
papertrading_model = xgb.XGBClassifier()
papertrading_model.load_model('xgb_clf.bst')
sclaer = pickle.load(open('scaler_model.pkl','rb'))



In [73]:
columns_model = ['HMA_5', 'RSI_5', 'ATR_14', 'RSI_14', 'RSI_150', 'cci']

In [74]:
alpaca_api = tradeapi.REST('','', 'https://paper-api.alpaca.markets')

In [84]:
data_trade = alpaca_api.get_crypto_bars(['BTC/USDT'], tradeapi.TimeFrame.Hour, "2023-01-01").df

In [85]:
for time in [5,14,150]:
    data_trade['RSI_'+str(time)] = TA.RSI(data_trade, time)
data_trade['ATR_14'] = TA.ATR(data_trade, 14)
data_trade['HMA_5'] = TA.HMA(data_trade, 5)
data_trade['cci'] = TA.CCI(data_trade)

In [86]:
data_to_predict = sclaer.transform(data_trade[columns_model].dropna())

In [146]:
quantity_to_buy = float(alpaca_api.get_position('USDTUSD').qty)/float(alpaca_api.get_latest_crypto_orderbook(['BTC/USDT'])['BTC/USDT'].bids[0].p)


In [145]:
alpaca_api.get_latest_crypto_orderbook(['BTC/USDT'])['BTC/USDT'].bids[0].p

20986.52579

In [147]:
quantity_to_buy

0.04748258274826081

In [177]:
def trade_in_alpaca(quantity, tries=0):
    if papertrading_model.predict(data_to_predict)[-1]==1 and papertrading_model.predict(data_to_predict)[-2]==0:
        try:
            alpaca_api.submit_order(symbol='BTC/USDT',qty=0.04,side="buy",type='market', time_in_force='ioc')
        except Exception as e:
            if 'insufficient balance' in str(e) and tries<=10:
                print(quantity, tries)
                trade_in_alpaca(quantity=0.9*quantity, tries=tries+1)
            else:
                print(e)

In [178]:
trade_in_alpaca(quantity=quantity_to_buy)

In [179]:
alpaca_api.list_positions()

[Position({   'asset_class': 'crypto',
     'asset_id': '2d8de014-3e11-499d-9578-59efc83522ba',
     'asset_marginable': False,
     'avg_entry_price': '1.00016629417383',
     'change_today': '-0.0001000100010001',
     'cost_basis': '996.691059344612548',
     'current_price': '0.9998',
     'exchange': 'FTXU',
     'lastday_price': '0.9999',
     'market_value': '996.3260379174028',
     'qty': '996.525342986',
     'qty_available': '996.525342986',
     'side': 'long',
     'symbol': 'USDTUSD',
     'unrealized_intraday_pl': '-0.36502142720971425525638',
     'unrealized_intraday_plpc': '-0.0003662332713707',
     'unrealized_pl': '-0.365021427209748',
     'unrealized_plpc': '-0.0003662332713707'})]